<a href="https://colab.research.google.com/github/Caarevalor/Python_Datascience_Notebooks/blob/main/More_Data_Processing_With_PANDAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fusionando Frames

In [ ]:
## En esta sección veremos como unir multiples DataFrame, ya sea uniendolo horizontalmente
#o concatenandolos verticalmente. Antes de abordar el codigo, debemos abordar una pequeña 
#parte de de teoría relacional.
#Comencemos pensando en un diagrama de Venn donde hay interseccion entre dos poblaciones
#entre estudiantes y personas que trabajan en la universidad, puede ser estudiantes que 
#trabajen como asistentes en distintas areas. Podemos pensar estas dos poblaciones como 
#índices en DataFrames separados, cuando queramos juntar estos dos dataframes debemos tomar
#algunas decisiones como si queremos toda la poblacion ya sea estudiantes o trabajor de la U
#Lo que sería una UNION en terminos de conjuntos, o en caso de que queramos solo aquellos casos
#en comun tendríamos una intersección

# Con esta vista previa comencemos a ver como se haría en PANDAS

import pandas as pd
#primero creemos dos dataFrames
staff_df=pd.DataFrame([{'Name':'Kelly', 'Role':'Director of HR'},
                       {'Name':'Sally', 'Role':'Course Liason'},
                       {'Name':'James', 'Role':'Grader'}])

#Ahora establezcamos los nombres como el indice del DataFrame
staff_df=staff_df.set_index('Name')

student_df=pd.DataFrame([{'Name':'Kelly', 'School':'Business'},
                       {'Name':'School', 'School':'Law'},
                       {'Name':'Sally', 'School':'Engineering'}])
student_df=student_df.set_index('Name')
print(staff_df.head())
print(student_df.head())

                 Role
Name                 
Kelly  Director of HR
Sally   Course Liason
James          Grader
             School
Name               
Kelly      Business
School          Law
Sally   Engineering


In [ ]:
#Como podemos observar hay dos estudiantes que estan en ambas listas, entonces 
#como los fusionamos? llamamos a merge(), en primer caso lo haremos con outer join

pd.merge(staff_df,student_df,how='outer',left_index=True,right_index=True )

,Role,School
Name,,
James,Grader,NaN
Kelly,Director of HR,Business
Sally,Course Liason,Engineering
School,NaN,Law


In [ ]:
pd.merge(staff_df,student_df,how='inner',left_index=True,right_index=True )

,Role,School
Name,,
Kelly,Director of HR,Business
Sally,Course Liason,Engineering


In [ ]:
#Tambien podemos poner el caso en que solo se quiera de staff_df detalles de 
#aquellos que sean estudiantes, para ello se usa left join
pd.merge(staff_df,student_df,how='left',left_index=True,right_index=True )

,Role,School
Name,,
Kelly,Director of HR,Business
Sally,Course Liason,Engineering
James,Grader,NaN


In [ ]:
#o podemos el caso contrario y usar un right join
pd.merge(staff_df,student_df,how='right',left_index=True,right_index=True )

,Role,School
Name,,
Kelly,Director of HR,Business
School,NaN,Law
Sally,Course Liason,Engineering


In [ ]:
#Tambien podemos unirlos no solo por indice, si no tambien por columna, hacuendo
#uso del parametro "on".
#Primero eliminemos el index
staff_df=staff_df.reset_index()
student_df=student_df.reset_index()

#Ahora fusionemos ambos con el parametro "on"
pd.merge(staff_df,student_df,how='right',on='Name')

,Name,Role,School
0,Kelly,Director of HR,Business
1,School,NaN,Law
2,Sally,Course Liason,Engineering


In [ ]:
#Y que sucede cuando hay conflicto entre ambos DataFrames?, echemos un vistazo
#añadiendo una nueva columna
staff_df=pd.DataFrame([{'Name':'Kelly', 'Role':'Director of HR',
                        'Location':'State Street'},
                       {'Name':'Sally', 'Role':'Course Liason',
                        'Location':'Washington Avenue'},
                       {'Name':'James', 'Role':'Grader',
                        'Location':'Washington Avenue'}])

student_df=pd.DataFrame([{'Name':'Kelly', 'School':'Business',
                          'Location':'1024 Billiard Avenue'},
                       {'Name':'School', 'School':'Law',
                        'Location':'Fraternity House #22'},
                       {'Name':'Sally', 'School':'Engineering',
                        'Location':'512 Wilson Crescent'}])
#En staff_df se refiere a la ubicación de la oficina donde se puede encontrar la 
#persona del staff, pero para student_df en realidad la ubicación hace referencia
#a la ubicacion de la casa.

#Merge conserva la información, pero depende de _x y _y para ayudar a diferenciar 
#con cual indice con cual columna de datos. El _x

#Si queremos toda la informacion del staff sean estudiantes o no, y de igual manera 
#para la informacion de estudiantes. Podemos realizar una left join con al columna
#nombres
pd.merge(staff_df, student_df,how='left',on='Name')

,Name,Role,Location_x,School,Location_y
0,Kelly,Director of HR,State Street,Business,1024 Billiard Avenue
1,Sally,Course Liason,Washington Avenue,Engineering,512 Wilson Crescent
2,James,Grader,Washington Avenue,NaN,NaN


In [ ]:
#Antes de pasar al siquiente tema, hablemos sobre multi-indexing y de columnas multiples
#Es bastante probable que el nombre del estudiantes y del staff se superponga, pero el 
#apellido no,en ese caso se pude utilizar una lista de multiples columnas para unir, 
#"keys" de ambos dataFrame usando el parametro On

staff_df=pd.DataFrame([{'First Name':'Kelly','Last Name':'Desjardins',
                        'Role':'Director of HR','Location':'State Street'},
                       {'First Name':'Sally', 'Last Name':'Brooks',
                        'Role':'Course Liason', 'Location':'Washington Avenue'},
                       {'First Name':'James', 'Last Name':'Wilde',
                        'Role':'Grader', 'Location':'Washington Avenue'}])

student_df=pd.DataFrame([{'First Name':'James','Last Name':'Hammond',
                          'School':'Business','Location':'1024 Billiard Avenue'},
                       {'First Name':'Mike', 'Last Name':'Smith',
                        'School':'Law','Location':'Fraternity House #22'},
                       {'First Name':'Sally','Last Name':'Brooks',
                        'School':'Engineering','Location':'512 Wilson Crescent'}])
##Se percata de que son James con diferentes apellidos:

pd.merge(staff_df,student_df,how='inner',on=['First Name','Last Name'])

,First Name,Last Name,Role,Location_x,School,Location_y
0,Sally,Brooks,Course Liason,Washington Avenue,Engineering,512 Wilson Crescent


In [ ]:
#Ahora vimos fusion horizontal, veamos para el caso de fusion vertical, haciendo uso de
#un par de bases:
#comenzamos haciendo uso de una magic function

%%capture 

df_2011 = pd.read_csv("datasets/college_scorecard/MERGED2011_12_PP.csv", error_bad_lines=False)
df_2012 = pd.read_csv("datasets/college_scorecard/MERGED2012_13_PP.csv", error_bad_lines=False)
df_2013 = pd.read_csv("datasets/college_scorecard/MERGED2013_14_PP.csv", error_bad_lines=False)

FileNotFoundError: ignored

In [ ]:
frames = [df_2011, df_2012, df_2013]
pd.concat(frames)

NameError: ignored

In [ ]:
#now that all the data is
# concatenated together, we don't know what observations are from what year anymore! Actually the concat
# function has a parameter that solves such problem with the keys parameter, we can set an extra level of
# indices, we pass in a list of keys that we want to correspond to the dataframes into the keys parameter

# Now let's try it out
pd.concat(frames, keys=['2011','2012','2013'])

# Pandas Idioms

Programadores de Python a menudo sugieren el uso del lenguaje para resolver problemas en particular, algunos más apropiados que otros, las más celebradas son Idiomatic Python, hay una gran cantidad de ejemplos en StackOverflow y otros sitios WEB.

Como un sub lenguaje dentro de Python, Pandas tiene su propio conjunto de modismos. Ya se ha aludido a ellos en anteriores secciones, como usar vectorización cuando sea posible, y uno usar bucles iterativos si no es necesarios.  Varios desarrroladores, y usuarios dentro de la comunidad de Pandas
usan el termino _pandorable_ para estos modismos(idioms). En esta sección se compartirá un par de claves para hacer el codigo más pandorable

In [ ]:
import pandas as pd
import numpy as np
#Y se traerá la funciones de tiempo para demostrar algo
import timeit
#Y cojamos el dataset sobre censo de USA
df=pd.read_csv('/content/drive/MyDrive/Python_for_Data_Science/Week_3/datasets/census.csv')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013,NPOPCHG_2014,NPOPCHG_2015,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015,NATURALINC2010,NATURALINC2011,NATURALINC2012,NATURALINC2013,NATURALINC2014,NATURALINC2015,INTERNATIONALMIG2010,...,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,GQESTIMATESBASE2010,GQESTIMATES2010,GQESTIMATES2011,GQESTIMATES2012,GQESTIMATES2013,GQESTIMATES2014,GQESTIMATES2015,RBIRTH2011,RBIRTH2012,RBIRTH2013,RBIRTH2014,RBIRTH2015,RDEATH2011,RDEATH2012,RDEATH2013,RDEATH2014,RDEATH2015,RNATURALINC2011,RNATURALINC2012,RNATURALINC2013,RNATURALINC2014,RNATURALINC2015,RINTERNATIONALMIG2011,RINTERNATIONALMIG2012,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RINTERNATIONALMIG2015,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,4801108,4816089,4830533,4846411,4858979,5034,15947,14981,14444,15878,12568,14226,59689,59062,57938,58334,58305,11089,48811,48357,50843,50228,50330,3137,10878,10705,7095,8106,7975,1357,...,677,-573,1135,116185,116212,115560,115666,116963,119088,119599,12.453020,12.282581,12.012080,12.056286,12.014973,10.183524,10.056360,10.541099,10.380963,10.371556,2.269496,2.226220,1.470981,1.675322,1.643417,1.027720,1.019840,1.002216,1.142716,1.179963,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,55253,55175,55038,55290,55347,89,593,-78,-137,252,57,151,636,615,574,623,600,152,507,558,583,504,467,-1,129,57,-9,119,133,33,...,22,-10,45,455,455,455,455,455,455,455,11.572789,11.138479,10.416194,11.293597,10.846281,9.225478,10.106133,10.579514,9.136393,8.442022,2.347311,1.032347,-0.163320,2.157204,2.404259,0.363924,0.289782,0.290347,0.326300,0.343466,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,186659,190396,195126,199713,203709,928,3466,3737,4730,4587,3996,517,2187,2092,2160,2186,2240,532,1825,1879,1902,2044,1992,-15,362,213,258,142,248,69,...,91,434,58,2307,2307,2307,2249,2304,2308,2309,11.826352,11.096524,11.205586,11.072868,11.104997,9.868812,9.966716,9.867141,10.353587,9.875515,1.957540,1.129809,1.338445,0.719281,1.229482,1.011215,0.912334,0.881921,1.073855,1.095627,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,27226,27159,26973,26815,26489,-116,-115,-67,-186,-158,-326,70,335,300,283,260,269,128,319,291,294,310,309,-58,16,9,-11,-50,-40,2,...,19,-1,-5,3193,3193,3382,3388,3389,3353,3352,12.278483,11.032454,10.455923,9.667584,10.093051,11.692048,10.701480,10.862337,11.526735,11.593877,0.586435,0.330974,-0.406414,-1.859151,-1.500825,-0.146609,-0.257424,-0.110840,-0.074366,0.000000,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,22733,22642,22512,22549,22583,-58,-128,-91,-130,37,34,44,266,245,259,247,253,34,278,237,281,211,223,10,-12,8,-22,36,30,2,...,14,-16,-21,2224,2224,2224,2224,2224,2233,2236,11.668202,10.798898,11.471852,10.962917,11.211557,12.194587,10.446281,12.446295,9.365083,9.882124,-0.526385,0.352617,-0.974443,1.597834,1.329434,0.438654,0.705234,0.797272,0.932070,0.930604,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [ ]:
#Lo primero que se tratará es el encadenamiento(chaining) en pandas. La idea general detrás del metodo Chaining,
#es que cada metodo sobre nuestro objeto devuelve una referencia a ese DataFrame. La belleza de esto es que se
#puede condensar muchas operaciones diferentes en un DataFrame, por ejemplo en una o dos lineas.

#Aqui es una manera pandorable de escribir codigo con el metodo encadenado. En esta parte se va a sacar el estado
#y las ciudades como indice multiple, y se va a hacer solo para datos que tiene un SUMLEV de 50, en este conjunto 
#de datos es del nivel de Condado. Se va a cambiar el nombre de la columna para que sea más legible.

(df.where(df['SUMLEV']==50)
  .dropna()
  .set_index(['STNAME','CTYNAME'])
  .rename(columns={'ESTIMATESDATABASE2010':'Estimate Data Base 2012'}))


SUMLEV  REGION  ...  RNETMIG2014  RNETMIG2015
STNAME  CTYNAME                            ...                          
Alabama Autauga County       50.0     3.0  ...     2.592270    -2.187333
        Baldwin County       50.0     3.0  ...    20.317142    18.293499
        Barbour County       50.0     3.0  ...    -3.978583   -10.543299
        Bibb County          50.0     3.0  ...     0.754533     1.107861
        Blount County        50.0     3.0  ...    -1.577232    -0.884411
...                           ...     ...  ...          ...          ...
Wyoming Sweetwater County    50.0     4.0  ...   -14.075283   -14.070195
        Teton County         50.0     4.0  ...    16.308671     1.520747
        Uinta County         50.0     4.0  ...   -14.740608   -12.606351
        Washakie County      50.0     4.0  ...   -18.020168     1.441961
        Weston County        50.0     4.0  ...     1.533635     6.935294

[3142 rows x 98 columns]

In [ ]:
#Aqui hay una manera más tradicional, no-pandorable de escribir lo anterior. No hay nada
#incorrecto en el sentido funcional, incluso tal vez sea más entendible para una persona
#nueva en este lenguaje. 

#Se crea un nuevo DataFrame a partir del original:
df= df[df['SUMLEV']==50]
#Ahora se actualizara el DataFrame para utilizar un nuevo index, usando inplace=True
df.set_index(['STNAME','CTYNAME'],inplace=True)
#Ahora modificar la columna nombres
df.rename(columns={'ESTIMATESBASE2010':'Estimates Base 2010'})

SUMLEV  REGION  ...  RNETMIG2014  RNETMIG2015
STNAME  CTYNAME                            ...                          
Alabama Autauga County         50       3  ...     2.592270    -2.187333
        Baldwin County         50       3  ...    20.317142    18.293499
        Barbour County         50       3  ...    -3.978583   -10.543299
        Bibb County            50       3  ...     0.754533     1.107861
        Blount County          50       3  ...    -1.577232    -0.884411
...                           ...     ...  ...          ...          ...
Wyoming Sweetwater County      50       4  ...   -14.075283   -14.070195
        Teton County           50       4  ...    16.308671     1.520747
        Uinta County           50       4  ...   -14.740608   -12.606351
        Washakie County        50       4  ...   -18.020168     1.441961
        Weston County          50       4  ...     1.533635     6.935294

[3142 rows x 98 columns]

In [ ]:
#Ahora la clave para cualquier buen "idiom" es entender que no está ayudando. En este 
#caso en realidad puede tomar el tiempo de ambos metodos para ver cual es mas rapido

#Podemos hacer una aproximacion y pasarlo a la funcion timeit, para tener una aproxima-
#cion y ver cual es mas rapida.

def first_approach():
  global df
  #y aquí ira nuestro codigo
  return (df.where(df['SUMLEV']==50)
            .dropna()
            .set_index(['STNAME','CTYNAME'])
            .rename(columns={'ESTIMATESDATABASE2010':'Estimate Data Base 2012'}))
df=pd.read_csv('/content/drive/MyDrive/Python_for_Data_Science/Week_3/datasets/census.csv')

timeit.timeit(first_approach,number=10)


0.21904491100002588

In [ ]:
# Ahora vamos probar con la segunda opcion. Como notará se hizo uso de una variable 
#global, modificar esta variable tendra su efecto incluso en un alcance global, y no 
#se quiere que eso suceda. Entonces antes de que eso suceda se creara otro df 

def second_approach():
  global df
  new_df= df[df['SUMLEV']==50]
  new_df.set_index(['STNAME','CTYNAME'],inplace=True)
  return new_df.rename(columns={'ESTIMATESBASE2010':'Estimates Base 2010'})

df=pd.read_csv('/content/drive/MyDrive/Python_for_Data_Science/Week_3/datasets/census.csv')
timeit.timeit(second_approach,number=10)

0.06364790500003892

In [ ]:
#Como se demuestra, la segunda accion es mucho más rapida, aunque normalmente se encuentren
#estos idioms en lugares como stack overflowm, hay que tener en cuenta que puede llegar a 
#generar problemas de rendimiento.

#Python tiene una gran herramienta, la funcion map, la cual es una base para la programación
#funcional del lenguaje. Cuando se usa map, se pasa una function que se quiere llamar y algun
#iterable, a la cual se desea que se aplique la función. Los resultados llaman cada elemento de
#la lista, y hay una lista resultado con la evaluacion de esa funcion.

#Pandas tiene algo similar, llamado applymap, la cual nos provee de una funcion que va a operar
#cada celda de un DataFrame, y me retorna el DataFrame en si mismo. Ya que generalmente uno requiere
#operar mas cada fila en vez de celda, se suele utilizar más apply. Veamos un ejemplo:

In [ ]:
#Tomemos nuevamente los datos de census.csv. Donde se encuentran 5 columnas interesantes, para las
#poblaciones estimadas, cada columna correspondiendo a un año. Es bastante razonable crear nuevas 
#columnas para los valores minimos y para los maximos, y con la función apply es sencillo de hacer

#Primero debemos escribir una funcion que tome una fila en particular, y encuentre el valor maximo
#y minimo, y nos retorne una nueva fila de datos a partir de lo anterior. Podemos crear un pequeño 
#slice de la fila proyectando las columnas de población. Luego podemos utilizar las funciones de 
#Numpy max() and min(), y crear una nueva serie  con los valores etiquetados que queremos aplicar.

def min_max(row):
  data=row[['POPESTIMATE2010',
            'POPESTIMATE2011',
            'POPESTIMATE2012',
            'POPESTIMATE2013',
            'POPESTIMATE2014',
            'POPESTIMATE2015']]
  return pd.Series({'min':np.min(data),'max':np.max(data)})

In [ ]:
#Ahora solo necesitamos llamar la función apply sobre el DataFrame.

#Apply toma la funcion, y los ejes sobre los cuales operar como parametros. Ahora hay 
#que ser un poco cuidadoso. Hemos hablados sobre los ejes zero son las filas del DataFrame
#en el pasado. Pero este parametro es realmente un parametro del indice a usar. Asi que,
#para aplicar en todas las filas, la cual se aplica a todas las columnas, puede pasar el eje
#eje igual a 1 o igual a 'columns'

df.apply(min_max, axis='columns').head()

,min,max
0,4785161,4858979
1,54660,55347
2,183193,203709
3,26489,27341
4,22512,22861


In [ ]:
#Aquí un ejemplo donde agregamos las dos nuevas columnas al dataFrame original

def min_max(row):
  data=row[['POPESTIMATE2010',
            'POPESTIMATE2011',
            'POPESTIMATE2012',
            'POPESTIMATE2013',
            'POPESTIMATE2014',
            'POPESTIMATE2015']]
  #Ahora creamos una nueva entrada para max
  row['max']=np.max(data)
  #Y una nueva entrada para min
  row['min']=np.min(data)
  return row
#Y ahora pasamos la funciona traves del DataFrame

df.apply(min_max,axis='columns')

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013,NPOPCHG_2014,NPOPCHG_2015,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015,NATURALINC2010,NATURALINC2011,NATURALINC2012,NATURALINC2013,NATURALINC2014,NATURALINC2015,INTERNATIONALMIG2010,...,RESIDUAL2015,GQESTIMATESBASE2010,GQESTIMATES2010,GQESTIMATES2011,GQESTIMATES2012,GQESTIMATES2013,GQESTIMATES2014,GQESTIMATES2015,RBIRTH2011,RBIRTH2012,RBIRTH2013,RBIRTH2014,RBIRTH2015,RDEATH2011,RDEATH2012,RDEATH2013,RDEATH2014,RDEATH2015,RNATURALINC2011,RNATURALINC2012,RNATURALINC2013,RNATURALINC2014,RNATURALINC2015,RINTERNATIONALMIG2011,RINTERNATIONALMIG2012,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RINTERNATIONALMIG2015,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,max,min
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,4801108,4816089,4830533,4846411,4858979,5034,15947,14981,14444,15878,12568,14226,59689,59062,57938,58334,58305,11089,48811,48357,50843,50228,50330,3137,10878,10705,7095,8106,7975,1357,...,1135,116185,116212,115560,115666,116963,119088,119599,12.453020,12.282581,12.012080,12.056286,12.014973,10.183524,10.056360,10.541099,10.380963,10.371556,2.269496,2.226220,1.470981,1.675322,1.643417,1.027720,1.019840,1.002216,1.142716,1.179963,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594,4858979,4785161
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,55253,55175,55038,55290,55347,89,593,-78,-137,252,57,151,636,615,574,623,600,152,507,558,583,504,467,-1,129,57,-9,119,133,33,...,45,455,455,455,455,455,455,455,11.572789,11.138479,10.416194,11.293597,10.846281,9.225478,10.106133,10.579514,9.136393,8.442022,2.347311,1.032347,-0.163320,2.157204,2.404259,0.363924,0.289782,0.290347,0.326300,0.343466,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333,55347,54660
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,186659,190396,195126,199713,203709,928,3466,3737,4730,4587,3996,517,2187,2092,2160,2186,2240,532,1825,1879,1902,2044,1992,-15,362,213,258,142,248,69,...,58,2307,2307,2307,2249,2304,2308,2309,11.826352,11.096524,11.205586,11.072868,11.104997,9.868812,9.966716,9.867141,10.353587,9.875515,1.957540,1.129809,1.338445,0.719281,1.229482,1.011215,0.912334,0.881921,1.073855,1.095627,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499,203709,183193
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,27226,27159,26973,26815,26489,-116,-115,-67,-186,-158,-326,70,335,300,283,260,269,128,319,291,294,310,309,-58,16,9,-11,-50,-40,2,...,-5,3193,3193,3382,3388,3389,3353,3352,12.278483,11.032454,10.455923,9.667584,10.093051,11.692048,10.701480,10.862337,11.526735,11.593877,0.586435,0.330974,-0.406414,-1.859151,-1.500825,-0.146609,-0.257424,-0.110840,-0.074366,0.000000,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299,27341,26489
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,22733,22642,22512,22549,22583,-58,-128,-91,-130,37,34,44,266,245,259,247,253,34,278,237,281,211,223,10,-12,8,-22,36,30,2,...,-21,2224,2224,2224,2224,2224,2233,2236,11.668202,10.798898,11.471852,10.962917,11.211557,12.194587,10.446281,12.446295,9.365083,9.882124,-0.526385,0.352617,-0.974443,1.597834,1.329434,0.438654,0.705234,0.797272,0.932070,0.930604,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861,22861,22512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
#Normalmente apply() no se usara con funciones tan grande, usualmente se suele
#utilizar con lambdas. Ejemplo:

rows=['POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013','POPESTIMATE2014','POPESTIMATE2015']
#ahora vamos aplicar esto a traves del dataframe con lambda

df.apply(lambda x:np.min(x[rows]),axis=1).head()

0    4785161
1      54660
2     183193
3      26489
4      22512
dtype: int64

In [ ]:
#tambien podriamos desarrollar una funcion que nos diga la region a la que pertenece Noreste, Medio Oeste, Sur y Oeste
# con informacion sacada de wikipedia. Ejemplo

def get_region(x):
    northeast = ['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 
                 'Rhode Island','Vermont','New York','New Jersey','Pennsylvania']
    midwest = ['Illinois','Indiana','Michigan','Ohio','Wisconsin','Iowa',
               'Kansas','Minnesota','Missouri','Nebraska','North Dakota',
               'South Dakota']
    south = ['Delaware','Florida','Georgia','Maryland','North Carolina',
             'South Carolina','Virginia','District of Columbia','West Virginia',
             'Alabama','Kentucky','Mississippi','Tennessee','Arkansas',
             'Louisiana','Oklahoma','Texas']
    west = ['Arizona','Colorado','Idaho','Montana','Nevada','New Mexico','Utah',
            'Wyoming','Alaska','California','Hawaii','Oregon','Washington']
    if x in northeast:
      return 'Northeast'            
    elif x in midwest:
      return 'Midwest'
    elif x in south:
      return 'South'
    else:
      return 'West'    

In [ ]:
#Ahora que ya tenemos la función podemos crear una nueva columna llamada region, la cuál mostrará
#la región del estado, podemos usar una funcion personalizada para ello. Una funcion personalizada
#se supone trabajara con la columna STNAME. Asi que estableceremos que la funcion se aplica sobre 
#esta columna.Ejemplo:

df['state_region']=df['STNAME'].apply(lambda x:get_region(x))

In [ ]:
df[['STNAME','state_region']].head()

,STNAME,state_region
0,Alabama,South
1,Alabama,South
2,Alabama,South
3,Alabama,South
4,Alabama,South


# Group by

A veces se quiere seleccionar un conjunto de datos agrupados, y entender los datos a nivel de grupo. Como hemos visto con pandas se puede iterar a traves de cada fila, pero es algo lento. Afortunadamente Pandas cuenta con una funcion llamada groupby() para acelerar estas tareas. La idea de esta funcion es que toma algun DataFrame, lo divide  en fragmentos basado en valores claves, y luego aplica el calculo en esos fragmentos, y combina el resultado dentro de otro DataFrame. En pandas esto es conocido como split-apply-combine.

## Splitting

In [ ]:
import pandas as pd
import numpy as np

#tomemos los datos de census.csv sobre USA

df= pd.read_csv('/content/drive/MyDrive/Python_for_Data_Science/Week_3/datasets/census.csv')
#Excluyamos algunos state level sumarization, con SUMLEVEL
df=df[df['SUMLEV']==50]
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,NPOPCHG_2010,NPOPCHG_2011,NPOPCHG_2012,NPOPCHG_2013,NPOPCHG_2014,NPOPCHG_2015,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,DEATHS2010,DEATHS2011,DEATHS2012,DEATHS2013,DEATHS2014,DEATHS2015,NATURALINC2010,NATURALINC2011,NATURALINC2012,NATURALINC2013,NATURALINC2014,NATURALINC2015,INTERNATIONALMIG2010,...,RESIDUAL2013,RESIDUAL2014,RESIDUAL2015,GQESTIMATESBASE2010,GQESTIMATES2010,GQESTIMATES2011,GQESTIMATES2012,GQESTIMATES2013,GQESTIMATES2014,GQESTIMATES2015,RBIRTH2011,RBIRTH2012,RBIRTH2013,RBIRTH2014,RBIRTH2015,RDEATH2011,RDEATH2012,RDEATH2013,RDEATH2014,RDEATH2015,RNATURALINC2011,RNATURALINC2012,RNATURALINC2013,RNATURALINC2014,RNATURALINC2015,RINTERNATIONALMIG2011,RINTERNATIONALMIG2012,RINTERNATIONALMIG2013,RINTERNATIONALMIG2014,RINTERNATIONALMIG2015,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,55253,55175,55038,55290,55347,89,593,-78,-137,252,57,151,636,615,574,623,600,152,507,558,583,504,467,-1,129,57,-9,119,133,33,...,22,-10,45,455,455,455,455,455,455,455,11.572789,11.138479,10.416194,11.293597,10.846281,9.225478,10.106133,10.579514,9.136393,8.442022,2.347311,1.032347,-0.163320,2.157204,2.404259,0.363924,0.289782,0.290347,0.326300,0.343466,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,186659,190396,195126,199713,203709,928,3466,3737,4730,4587,3996,517,2187,2092,2160,2186,2240,532,1825,1879,1902,2044,1992,-15,362,213,258,142,248,69,...,91,434,58,2307,2307,2307,2249,2304,2308,2309,11.826352,11.096524,11.205586,11.072868,11.104997,9.868812,9.966716,9.867141,10.353587,9.875515,1.957540,1.129809,1.338445,0.719281,1.229482,1.011215,0.912334,0.881921,1.073855,1.095627,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,27226,27159,26973,26815,26489,-116,-115,-67,-186,-158,-326,70,335,300,283,260,269,128,319,291,294,310,309,-58,16,9,-11,-50,-40,2,...,19,-1,-5,3193,3193,3382,3388,3389,3353,3352,12.278483,11.032454,10.455923,9.667584,10.093051,11.692048,10.701480,10.862337,11.526735,11.593877,0.586435,0.330974,-0.406414,-1.859151,-1.500825,-0.146609,-0.257424,-0.110840,-0.074366,0.000000,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,22733,22642,22512,22549,22583,-58,-128,-91,-130,37,34,44,266,245,259,247,253,34,278,237,281,211,223,10,-12,8,-22,36,30,2,...,14,-16,-21,2224,2224,2224,2224,2224,2233,2236,11.668202,10.798898,11.471852,10.962917,11.211557,12.194587,10.446281,12.446295,9.365083,9.882124,-0.526385,0.352617,-0.974443,1.597834,1.329434,0.438654,0.705234,0.797272,0.932070,0.930604,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,57711,57776,57734,57658,57673,51,338,65,-42,-76,15,183,744,710,646,618,603,133,570,592,585,589,590,50,174,118,61,29,13,5,...,-22,-14,53,489,489,489,489,489,489,489,12.929686,12.295756,11.185179,10.711314,10.456859,9.905808,10.252236,10.128993,10.208680,10.231421,3.023878,2.043520,1.056186,0.502634,0.225438,0.052136,0.329041,0.346290,0.485302,0.485559,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411


In [ ]:
#Para el primer ejemplo de groupby(), se quiere utilizar el census date. Comencemos
#por tomar una lista de los estados unicos. Podemos iterar sobre cada estado para 
#reducir el data frame y calcular el promedio

#vamos a calcular estos tres veces, haciendo uso de una cell magic function %%timeit
%%timeit -n 3
for state in df['STNAME'].unique():
  #calculemos el promedio haciendo uso de numpy
  avg=np.average(df.where(df['STNAME']==state).dropna()['CENSUS2010POP'])
  print('Counties in state '+ state +
        ' have an avarage population of '+ str(avg))

Counties in state Alabama have an avarage population of 71339.34328358209
Counties in state Alaska have an avarage population of 24490.724137931036
Counties in state Arizona have an avarage population of 426134.4666666667
Counties in state Arkansas have an avarage population of 38878.90666666667
Counties in state California have an avarage population of 642309.5862068966
Counties in state Colorado have an avarage population of 78581.1875
Counties in state Connecticut have an avarage population of 446762.125
Counties in state Delaware have an avarage population of 299311.3333333333
Counties in state District of Columbia have an avarage population of 601723.0
Counties in state Florida have an avarage population of 280616.5671641791
Counties in state Georgia have an avarage population of 60928.63522012578
Counties in state Hawaii have an avarage population of 272060.2
Counties in state Idaho have an avarage population of 35626.86363636364
Counties in state Illinois have an avarage populat

In [ ]:
#Ahora hagamos una aproximacion usando un groupby()

%%timeit -n 3
#Para empezar vamos a decirle a Pandas que queremos agrupar el estado por nombre

for group, frame in df.groupby('STNAME'):
  #Como se podrá notar hay dos valores que se establecen, groupby retorna una tupla, donde el primer
  #valor es la clave(key) por el cual tratamos de agrupar, este caso el State Name, y el segundo valor
  #es el dataFrame proyectado que fue encontrado para este grupo. 

  #Ahora vamos a incluir la logica para calcular el promedio del censo en 2010
  avg=np.average(frame['CENSUS2010POP'])
  print('Counties in state '+ group +
        ' have an avarage population of '+ str(avg))

Counties in state Alabama have an avarage population of 71339.34328358209
Counties in state Alaska have an avarage population of 24490.724137931036
Counties in state Arizona have an avarage population of 426134.4666666667
Counties in state Arkansas have an avarage population of 38878.90666666667
Counties in state California have an avarage population of 642309.5862068966
Counties in state Colorado have an avarage population of 78581.1875
Counties in state Connecticut have an avarage population of 446762.125
Counties in state Delaware have an avarage population of 299311.3333333333
Counties in state District of Columbia have an avarage population of 601723.0
Counties in state Florida have an avarage population of 280616.5671641791
Counties in state Georgia have an avarage population of 60928.63522012578
Counties in state Hawaii have an avarage population of 272060.2
Counties in state Idaho have an avarage population of 35626.86363636364
Counties in state Illinois have an avarage populat

In [ ]:
##Como se puede notar la diferencia es muy grande, en el 99% de los casos se utilizaran
#para agrupar una o dos columnas. Pero tambien se puede usar para segmentar

#Se desea trabajar solo en una tercera parte de los estados en un momento dado. Podríamos crear alguna función 
#que devuelva un número entre cero y dos basado en el primer carácter del nombre del estado. Entonces podemos 
#decirle a group by que use esta función para dividir nuestro marco de datos. Es importante tener en cuenta que
#para hacer esto, debe configurar el índice del marco de datos para que sea la columna por la que desea agrupar primero.

#Crearemos una nueva función llamada set_batch_number y si la primera letra del parámetro es una M mayúscula, 
#devolveremos un 0. Si es una Q mayúscula, devolveremos un 1 y, de lo contrario, devolveremos un 2.
#Entonces Pasaremos esta función al marco de datos

df=df.set_index('STNAME')
def set_batch_number(item):
  if item[0]<'M':
    return 0
  if item[0]>'Q':
    return 1
  else:
    return 2
for group, frame in df.groupby(set_batch_number):
  print('There are ' + str(len(frame))+ ' records in group '+ str(group)+' for processing.')

There are 1177 records in group 0 for processing.
There are 831 records in group 1 for processing.
There are 1134 records in group 2 for processing.


In [ ]:
#Vamos a realizar otro ejemplo sobre como usar un agrupado de datos con un dataset de
#airbnb. En este dataset hay dos conlumnas de interes, una es la politica de cancelacion
#y el otro es las puntuaciones de las reseñas(review scores value). Empecemos==>

df=pd.read_csv('/content/drive/MyDrive/Python_for_Data_Science/Week_3/datasets/listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,...,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",NaN,"The bus stop is 2 blocks away, and frequent. B...","You will have access to 2 bedrooms, a living r...",NaN,Clean up and treat the home the way you'd like...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,31303940,https://www.airbnb.com/users/show/31303940,Virginia,2015-04-15,"Boston, Massachusetts, United States",We are country and city connecting in our deck...,NaN,NaN,NaN,f,https://a2.muscache.com/im/pictures/5936fef0-b...,https://a2.muscache.com/im/pictures/5936fef0-b...,Roslindale,1,1,"['email', 'phone', 'facebook', 'reviews']",t,f,"Birch Street, Boston, MA 02131, United States",Roslindale,Roslindale,...,2.0,3.0,Real Bed,"{TV,""Wireless Internet"",Kitchen,""Free Parking ...",NaN,$250.00,NaN,NaN,NaN,$35.00,1,$0.00,2,1125,2 weeks ago,NaN,0,0,0,0,2016-09-06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...","If you don't have a US cell phone, you can tex...",Plenty of safe street parking. Bus stops a few...,Apt has one more bedroom (which I use) and lar...,"If I am at home, I am likely working in my hom...",Pet friendly but please confirm with me if the...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,2572247,https://www.airbnb.com/users/show/2572247,Andrea,2012-06-07,"Boston, Massachusetts, United States",I live in Boston and I like to travel and have...,within an hour,100%,100%,f,https://a2.muscache.com/im/users/2572247/profi...,https://a2.muscache.com/im/users/2572247/profi...,Roslindale,1,1,"['email', 'phone', 'facebook', 'linkedin', 'am...",t,t,"Pinehurst Street, Boston, MA 02131, United States",Roslindale,Roslindale,...,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Air Conditio...",NaN,$65.00,$400.00,NaN,$95.00,$10.00,0,$0.00,2,15,a week ago,NaN,26,54,84,359,2016-09-06,36,2014-06-01,2016-08-13,94.0,10.0,9.0,10.0,10.0,9.0,9.0,f,NaN,NaN,t,moderate,f,f,1

In [ ]:
#Como se agruparía por estas dos columnas? Un primer enfoque sería promover estas dos a 
#multi-index, y luego llamar groupby().

df=df.set_index(['cancellation_policy','review_scores_value'])

#Cuando se tiene un multi-indice, se tiene que pasar los niveles en los que estamos interesados
#en agrupar. De forma predeterminada groupby() no sabe que se desea agrupar por niveles

for group , frame in df.groupby(level=(0,1)):
  print (group)

('flexible', 2.0)
('flexible', 4.0)
('flexible', 5.0)
('flexible', 6.0)
('flexible', 7.0)
('flexible', 8.0)
('flexible', 9.0)
('flexible', 10.0)
('moderate', 2.0)
('moderate', 4.0)
('moderate', 6.0)
('moderate', 7.0)
('moderate', 8.0)
('moderate', 9.0)
('moderate', 10.0)
('strict', 2.0)
('strict', 3.0)
('strict', 4.0)
('strict', 5.0)
('strict', 6.0)
('strict', 7.0)
('strict', 8.0)
('strict', 9.0)
('strict', 10.0)
('super_strict_30', 6.0)
('super_strict_30', 7.0)
('super_strict_30', 8.0)
('super_strict_30', 9.0)
('super_strict_30', 10.0)


In [ ]:
#Parece que lo anterior funciona bien, pero que si quisieramos agrupar por cancellation policy y review scores,
#pero separando los puntajes de 10, de todos aquellos debajo de 10? . Ejemplo==>

def grouping_fun(item):
  #Comprobemos la parte de "review_scores_value" de el index, item esta en el formato de 
  #(cancellation_policy, review_scores_value)
  if item[1]==10.0:
    return (item[0],"10.0")
  else:
    return (item[0],'Not 10.0')
for group, frame in df.groupby(by=grouping_fun):
  print(group)   

('flexible', '10.0')
('flexible', 'Not 10.0')
('moderate', '10.0')
('moderate', 'Not 10.0')
('strict', '10.0')
('strict', 'Not 10.0')
('super_strict_30', '10.0')
('super_strict_30', 'Not 10.0')


In [ ]:
df.head()

id  ... reviews_per_month
cancellation_policy review_scores_value            ...                  
moderate            NaN                  12147973  ...               NaN
                    9.0                   3075044  ...              1.30
                    10.0                     6976  ...              0.47
                    10.0                  1436513  ...              1.00
flexible            10.0                  7651065  ...              2.25

[5 rows x 93 columns]

In [ ]:
#Hasta el momento se ha hecho un procesamiento simple sobre el splitting basicamente solo mostrar como
#quedan los splits del dataframe. Pandas tene desarrrolladores en tres grandes categorias de procesamiento
#de datos que suceden durante el paso de aplicación, "Agregacion de datos de grupo", "Transformacion de 
#grupos de datos", y "Filtracion de grupos de datos"

## Agreggation

In [ ]:
#El paso mas sencillo es el de agregación, el cual utiliza el metodo agg, sobre un objeto groupby.
#Hasta el momento solo se ha iteado sobre a traves de un objeto groupby, desempacandolo en una etiqueta,
#y un DataFrame. Pero con agg podemos pasar un diccionario de las columnas en las que se esta interesado
#en agregar junto a la funció que se esta buscando aplicar. 

#Reiniciemos el index de los datos de airbnb

df=df.reset_index()

#Ahora vamos a grupar por cancellation_policy y encontrar el promedio de review_scores_value por grupo

df.groupby("cancellation_policy").agg({"review_scores_value":np.nanmean})

,review_scores_value
cancellation_policy,
flexible,9.237421
moderate,9.307398
strict,9.081441
super_strict_30,8.537313


In [ ]:
df.groupby("cancellation_policy").agg('mean')['review_scores_value']

cancellation_policy
flexible           9.237421
moderate           9.307398
strict             9.081441
super_strict_30    8.537313
Name: review_scores_value, dtype: float64

In [ ]:
#Lo anterior se puede extender a varias funciones
df.groupby("cancellation_policy").agg(([('average','mean'),('Desviacion','std')]))['review_scores_value'].agg

<bound method DataFrame.aggregate of                       average  Desviacion
cancellation_policy                      
flexible             9.237421    1.096271
moderate             9.307398    0.859859
strict               9.081441    1.040531
super_strict_30      8.537313    0.840785>

In [ ]:
df.groupby("cancellation_policy").agg({"review_scores_value":(np.nanmean,np.nanstd),
                                      "reviews_per_month":np.nanmean})

review_scores_value           reviews_per_month
                                nanmean    nanstd           nanmean
cancellation_policy                                                
flexible                       9.237421  1.096271          1.829210
moderate                       9.307398  0.859859          2.391922
strict                         9.081441  1.040531          1.873467
super_strict_30                8.537313  0.840785          0.340143

## Transformation

In [ ]:
#La transformacion es diferente  de "Aggregation", donde agg() devuelve un solo valor por columna, asi un una fila
#por grupo, transform() devuelve un objeto que es del mismo tamaño del objeto. Esencialmente, transmite una funcion
#que nosostros le suministramos a traves del DataFrame del grupo, devolviendo un nuevo marco de datos. Esto hace que
#la combinacion de datos mas adelante sea más sencilla.

#Por ejemplo, supongamos que queríamos incluir los valores de calificación promedio en un grupo determinado por 
#cancellation_policy, pero preservar la forma(shape) del DataFrame de manera que podamos preservar una diferencia 
#entre la observacion invidual y la suma:

#Primero se define el subconjunto de columnas que son de interes:

cols=['cancellation_policy','review_scores_value']

#Ahora pasemos a transformarlo. Se guardará dentro del mismo DataFrame:

transform_df= df[cols].groupby('cancellation_policy').transform(np.nanmean)
transform_df.head()

,review_scores_value
0,9.307398
1,9.307398
2,9.307398
3,9.307398
4,9.237421


In [ ]:
#se observa que el indice es el mismo que en DataFrame original. Vamos a unirlo, pero antes de eso,
#vamos a renombrar la columna transformada:

transform_df.rename({'review_scores_value':'mean_review_scores'},axis='columns',inplace=True)
df=df.merge(transform_df,left_index=True,right_index=True)
df.head()

,cancellation_policy,review_scores_value,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,...,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,requires_license,license,jurisdiction_names,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores
0,moderate,NaN,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",NaN,"The bus stop is 2 blocks away, and frequent. B...","You will have access to 2 bedrooms, a living r...",NaN,Clean up and treat the home the way you'd like...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,31303940,https://www.airbnb.com/users/show/31303940,Virginia,2015-04-15,"Boston, Massachusetts, United States",We are country and city connecting in our deck...,NaN,NaN,NaN,f,https://a2.muscache.com/im/pictures/5936fef0-b...,https://a2.muscache.com/im/pictures/5936fef0-b...,Roslindale,1,1,"['email', 'phone', 'facebook', 'reviews']",t,f,"Birch Street, Boston, MA 02131, United States",...,1.5,2.0,3.0,Real Bed,"{TV,""Wireless Internet"",Kitchen,""Free Parking ...",NaN,$250.00,NaN,NaN,NaN,$35.00,1,$0.00,2,1125,2 weeks ago,NaN,0,0,0,0,2016-09-06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,f,f,1,NaN,9.307398
1,moderate,9.0,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...","If you don't have a US cell phone, you can tex...",Plenty of safe street parking. Bus stops a few...,Apt has one more bedroom (which I use) and lar...,"If I am at home, I am likely working in my hom...",Pet friendly but please confirm with me if the...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,2572247,https://www.airbnb.com/users/show/2572247,Andrea,2012-06-07,"Boston, Massachusetts, United States",I live in Boston and I like to travel and have...,within an hour,100%,100%,f,https://a2.muscache.com/im/users/2572247/profi...,https://a2.muscache.com/im/users/2572247/profi...,Roslindale,1,1,"['email', 'phone', 'facebook', 'linkedin', 'am...",t,t,"Pinehurst Street, Boston, MA 02131, United States",...,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Air Conditio...",NaN,$65.00,$400.00,NaN,$95.00,$10.00,0,$0.00,2,15,a week ago,NaN,26,54,84,359,2016-09-06,36,2014-06-01,2016-08-13,94.0,10.0,9.0,10.0,10.0,9.0,f,NaN,NaN,t,f,f,1,1.30,9.307398
2,moderate,10.0,6976

In [ ]:
#Ahora se podría crear una columna que tenga la diferencia entre una fila dada y su grupo(cancellation_policy.means)
df['mean_diff']=np.absolute(df['review_scores_value']-df['mean_review_scores'])
df['mean_diff'].head()

0         NaN
1    0.307398
2    0.692602
3    0.692602
4    0.762579
Name: mean_diff, dtype: float64

## Filtering

In [ ]:
#Por lo tanto el objeto groupby tambien es compatible con grupos de filtrado. Es frecuente que se quiera agrupar por
#determinadas caracteristicas, luego hacer alguna transformacion sobre esos grupos, y luego eliminar ciertos grupos
#como parte  de una rutina de limpieza. La funcion filter() toma una funcion que se aplica a cada grupo del DataFrame
#y retorna un verdadero o falso, dependiendo de si se debe incluir o no en los resultados. 

#Por ejempo si se quisiera un grupo de aquellso tienen una calificacion superior a 9. Ejemplo:

df.groupby('cancellation_policy').filter(lambda x: np.nanmean(x['review_scores_value'])>9.2)

,cancellation_policy,review_scores_value,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,...,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,requires_license,license,jurisdiction_names,instant_bookable,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,mean_review_scores,mean_diff
0,moderate,NaN,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",NaN,"The bus stop is 2 blocks away, and frequent. B...","You will have access to 2 bedrooms, a living r...",NaN,Clean up and treat the home the way you'd like...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,31303940,https://www.airbnb.com/users/show/31303940,Virginia,2015-04-15,"Boston, Massachusetts, United States",We are country and city connecting in our deck...,NaN,NaN,NaN,f,https://a2.muscache.com/im/pictures/5936fef0-b...,https://a2.muscache.com/im/pictures/5936fef0-b...,Roslindale,1,1,"['email', 'phone', 'facebook', 'reviews']",t,f,"Birch Street, Boston, MA 02131, United States",...,2.0,3.0,Real Bed,"{TV,""Wireless Internet"",Kitchen,""Free Parking ...",NaN,$250.00,NaN,NaN,NaN,$35.00,1,$0.00,2,1125,2 weeks ago,NaN,0,0,0,0,2016-09-06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,f,f,1,NaN,9.307398,NaN
1,moderate,9.0,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...","If you don't have a US cell phone, you can tex...",Plenty of safe street parking. Bus stops a few...,Apt has one more bedroom (which I use) and lar...,"If I am at home, I am likely working in my hom...",Pet friendly but please confirm with me if the...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,2572247,https://www.airbnb.com/users/show/2572247,Andrea,2012-06-07,"Boston, Massachusetts, United States",I live in Boston and I like to travel and have...,within an hour,100%,100%,f,https://a2.muscache.com/im/users/2572247/profi...,https://a2.muscache.com/im/users/2572247/profi...,Roslindale,1,1,"['email', 'phone', 'facebook', 'linkedin', 'am...",t,t,"Pinehurst Street, Boston, MA 02131, United States",...,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Air Conditio...",NaN,$65.00,$400.00,NaN,$95.00,$10.00,0,$0.00,2,15,a week ago,NaN,26,54,84,359,2016-09-06,36,2014-06-01,2016-08-13,94.0,10.0,9.0,10.0,10.0,9.0,f,NaN,NaN,t,f,f,1,1.30,9.307398,0.307398
2,moderate,10.0

In [ ]:
#Observar que el resultado todavia esta indexado, pero cualquiera de los resultados que estaba en el grupo con la
#puntuacion de review menor a 9.2 no estan copiados



## Applying

In [ ]:
#Por lejos, la operación mas común que se invoca al agrupar es la funcion apply(). Esto nos permite aplicar una 
#funcion arbitraria a cada grupo, y cocer los resultados de vuelta en un solo DataFrame, donde el indice es preservado

#Ejemplo con airbnb:

df=pd.read_csv('/content/drive/MyDrive/Python_for_Data_Science/Week_3/datasets/listings.csv')
 
# Y ahora incluimos algunas columnas en las que se esta interesado

df=df[['cancellation_policy','review_scores_value']]
df.head()

,cancellation_policy,review_scores_value
0,moderate,NaN
1,moderate,9.0
2,moderate,10.0
3,moderate,10.0
4,flexible,10.0


In [ ]:
#En trabajos anteriores, se quería encontrar la puntuacion media de un listado, su desviacion media del grupo.
#Esto fue un proceso de dos pasos, primero se usó transform() en un objeto groupby, y luego se tuvo que transmitir
#para crear una nueva columna. Con Apply() esto se puede realizar en una sola linea

def calc_mean_review_scores(group):
  #group es el dataFrame que hemos agrupado
  avg=np.nanmean(group['review_scores_value'])
  #Y ahora nuestra formula para crear una nueva columna
  group['review_scores_mean']=np.abs(avg-group['review_scores_value'])
  return group

#Ahora es solo aplicar esto a los grupos

df.groupby('cancellation_policy').apply(calc_mean_review_scores).head()

,cancellation_policy,review_scores_value,review_scores_mean
0,moderate,NaN,NaN
1,moderate,9.0,0.307398
2,moderate,10.0,0.692602
3,moderate,10.0,0.692602
4,flexible,10.0,0.762579


Usar apply() puede ser **más lento** que algunas funciones especializadas, especialmente **agg()**. Pero si los DataFrame no son enormes. Es una buena manera solida de hacerlo.

# Scales

Hay al menos 4 escalas:
1. Escala de proporciones:
  - Las unidades estan igualmente espaciadas
  - Las operaciones matematicas como -+*/ son validas
  - En ejemplo es altura y peso
2. Escala de intervalos:
  - Las unidades tambien estan ingualmente espaciadas, pero no hay valor que en verdad sea 0
  - Como no hay 0, operaciones como * o / no son validas
  - Ejemplo la temperatura no existe la asuencia de temperatura como tal, 0 ° tiene un significado como tal o una brujula. 
3. Escala ordinal:
  - El orden entre unidades es muy importante, pero los valores no están igualmente espaciadas.
  - La calificacion por letras es un buen ejemplo A,A+,B...
4. Escala nominal:
  - Datos categoricos, pero las categorias no tienen orden respecto a otros.
  - Por ejemplo a que el nombre de un equipo.

¿Por que hablar de esto ahora? Panda tiene herramientas para trabajar con este tipos de datos, por ejemplo datos categoricos, podemos asignarlos como una variable de este tipo con astype.

In [ ]:
import pandas as pd
#vamos a crear  DataFrame con calficiaciones de letras, en orden descendente. Tambien se asignará un indice y
#aquí se hará que sea el juicio humano de que tan bueno era un estudiante, como "excelente" o "good"
df=pd.DataFrame(['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D'],
                index=['excellent', 'excellent', 'excellent', 'good', 'good', 'good', 
                       'ok', 'ok', 'ok', 'poor', 'poor'],
                columns=['Grades'])
df

,Grades
excellent,A+
excellent,A
excellent,A-
good,B+
good,B
good,B-
ok,C+
ok,C
ok,C-
poor,D+


In [ ]:
#si se revisamos el tipo de dato que tiene la columna, nos retornará que es un objeto:

df.dtypes

Grades    object
dtype: object

In [ ]:
#Sin embargo,se puede decirle a pandas que se desea cambiar el tipo de dato de la caregoría
#haciendo uso de astype():

df['Grades'].astype("category").head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): ['A', 'A+', 'A-', 'B', ..., 'C+', 'C-', 'D', 'D+']

In [ ]:
#Ahora se muestra que hay 11 categorías, y los pandas son conscientes de lo que son esas categorías, 
#Sin embargo, lo más interesante es que nuestros datos no son solo en estan categorizados, sino que en realidad
#no estan ordenados, si no que en realidad ya esta ordenado. Tambien se le puede decir a pandas que los datos 
#se ordenan creando primero un nuevo tipo de datos caregoricos con la lista de categorías en orden, y el orden
#es igual a la bandera verdadera(ordered=True). 

my_categories=pd.CategoricalDtype(categories=['D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+'], 
                           ordered=True)
grades=df['Grades'].astype(my_categories)
grades.head()

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']

In [ ]:
#Ahora que ya reconoce el orden de las categorías, esto puede ayudar a la hora de realizar un enmascaramiento
#booleano. Como por ejemplo obtener solo los estudiantes con una nota mayor a c. Ejemplo:

df[df['Grades']>'C']

,Grades
ok,C+
ok,C-
poor,D+
poor,D


In [ ]:
#Como se observa C+ es mayor que C, pero el resto no, entonces lo que hacemos es transmitir a traves del 
#DataFrame que tiene establecido el orden categorico.
grades[grades>'C']

excellent    A+
excellent     A
excellent    A-
good         B+
good          B
good         B-
ok           C+
Name: Grades, dtype: category
Categories (11, object): ['D' < 'D+' < 'C-' < 'C' ... 'B+' < 'A-' < 'A' < 'A+']

In [ ]:
# a veces es util convertir variables categoricas en variables ficticias(dummies variables).
# es cuando variables categoricas toman valores de 0 o 1. Afortunadamente Pandas cuenta con 
#con una funcion llmada get_dummies() que convierte una columna en multiples columnas de 0
#y 1, indicando la presencia de una variable ficticia. Rara vez se usa, pero cuando se utiliza
#es muy útil. 

In [ ]:
#Hay un funcion más basada en Scale operation, que nos gustaría introducir, y es convertir una
#escala de algo que es la escala de intervalo o ratio, como un grado numero, en uno categorico
#Por ejemplo, si se esta viendo frecuencia de categorias, este puede ser un enfoque extremadamente
#utíl,y los histogramas se suslen en  intervalos. Además si se está usando machine learning classification
#se deberá utilizar datos categoricos. Reduciendo la dimensionalidad puede ser util para aplicar una tecnica
#dada. Pandas tiene una función llamada "cut()" que toma como argumento una estructura similar a una matriz,
#como una columna, DataFrame, Series. Y tambien toma varios compartimentos(bins) para ser utilizados, y 
# todos los compartimentos se mantienen igualmente espaciados. 

#Volvamos a algunos datos de census.csv, se observo que se podían agrupar por estado y luego agregar una lista
#del tamaño promedio del condado por estado. Si aplicamos cut() con digamos 10 compartimentos(bins), podríamos
#ver los estados como categoricos, utilizando el tamaño promedio del condado. Ejemplo:

import numpy as np
df=pd.read_csv('/content/drive/MyDrive/Python_for_Data_Science/Week_3/datasets/census.csv')

# y reducimos el nivel de los datos country

df=df[df['SUMLEV']==50]

#y ahora por grupos

df=df.set_index('STNAME').groupby(level=0)['CENSUS2010POP'].agg(np.average)

df.head()

STNAME
Alabama        71339.343284
Alaska         24490.724138
Arizona       426134.466667
Arkansas       38878.906667
California    642309.586207
Name: CENSUS2010POP, dtype: float64

In [ ]:
#ahora si se desea hacer "BINS" de cada uno de ellos, se usa cut()

pd.cut(df,10)
#esta es solo una manera de crear datos categoricos, cut() nos da cateorias igualmente
#espaciadas basadas en la frencuencia.

STNAME
Alabama                   (11706.087, 75333.413]
Alaska                    (11706.087, 75333.413]
Arizona                 (390320.176, 453317.529]
Arkansas                  (11706.087, 75333.413]
California              (579312.234, 642309.586]
Colorado                 (75333.413, 138330.766]
Connecticut             (390320.176, 453317.529]
Delaware                (264325.471, 327322.823]
District of Columbia    (579312.234, 642309.586]
Florida                 (264325.471, 327322.823]
Georgia                   (11706.087, 75333.413]
Hawaii                  (264325.471, 327322.823]
Idaho                     (11706.087, 75333.413]
Illinois                 (75333.413, 138330.766]
Indiana                   (11706.087, 75333.413]
Iowa                      (11706.087, 75333.413]
Kansas                    (11706.087, 75333.413]
Kentucky                  (11706.087, 75333.413]
Louisiana                 (11706.087, 75333.413]
Maine                    (75333.413, 138330.766]
Maryland     

# Pivot Table

Una tabla dinamica(**Pivot Table**) es una forma de resumir un DataFrame por motivos particulares. Hace un fuerte uso de la funcion aggregation. Una tabla dinamica es en si misma un dataFrame, cuando las variables representan una variable en la que se esta interesado y una columna otra, y las celdas dan un valor agregado. Una tabla dinamica tiende tambien a incluir valores marginales, las cuales son sumas de cada columna y fila. Esto permite ver la relación entre dos variables de un vistazo. 

In [ ]:
#Vamos a echarles un vistazo en pandas
import pandas as pd
import numpy as np

#aqui un conjunto de datos(dataset) de "Times Higher Education World  University Ranking"

df= pd.read_csv('/content/drive/MyDrive/Python_for_Data_Science/Week_3/datasets/cwurData.csv')
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [ ]:
#Supongamos que se desea crear una nueva columna llamda 'Rank_Level', donde las instituciones
#con clasificacion mundial 1-100 serán categorizadas como "first tier", y las 101-200 "second tier",
#201-300 "third tier", y despues de 301 como otra categoria de universidades top.

def create_category(ranking):
  if (ranking >= 1) & (ranking <=100):
    return "First Tier Top Ranking"
  elif (ranking > 100) & (ranking <=200):
    return "Second Tier Top Ranking"
  elif (ranking > 200) & (ranking <=300):
    return "Third Tier Top Ranking"
  else:
    return "Other Top University"

#Ahora utilizamos la funcion apply() sobre est columna para crear una nueva Series
df['Rank_Level']=df['world_rank'].apply(lambda x:create_category(x))

df.head()

#df[df['country']==''].shape

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,Rank_Level
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,First Tier Top Ranking
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,First Tier Top Ranking
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,First Tier Top Ranking
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,First Tier Top Ranking
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,First Tier Top Ranking


In [ ]:
#Por tanto una tabla dinamica nos permite desplazar una de estas columnas en nuevos encabezados de columna y compararla, 
#con otra columna como indices de fila. Por ejemplo se quiere comparar en rank level vs country de las universidades en 
#terminos de puntuación general

#Para realizar esto se requieren del puntaje y el indice(index) del pais, y que las columnas sean los niveles de rango.

df.pivot_table(values='score',index='country',columns='Rank_Level',aggfunc=[np.mean]).head()

mean  ...                       
Rank_Level First Tier Top Ranking  ... Third Tier Top Ranking
country                            ...                       
Argentina                     NaN  ...                    NaN
Australia                 47.9425  ...              47.285000
Austria                       NaN  ...              47.066667
Belgium                   51.8750  ...              46.746667
Brazil                        NaN  ...                    NaN

[5 rows x 4 columns]

In [ ]:
df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.median], margins=True)

median  ...        
Rank_Level           First Tier Top Ranking  ...     All
country                                      ...        
Argentina                               NaN  ...  44.390
Australia                            48.055  ...  44.765
Austria                                 NaN  ...  44.690
Belgium                              51.875  ...  46.210
Brazil                                  NaN  ...  44.380
Bulgaria                                NaN  ...  44.335
Canada                               53.920  ...  45.620
Chile                                   NaN  ...  44.805
China                                53.430  ...  44.440
Colombia                                NaN  ...  44.410
Croatia                                 NaN  ...  44.770
Cyprus                                  NaN  ...  44.210
Czech Republic                          NaN  ...  44.440
Denmark                              49.815  ...  47.550
Egypt                                   NaN  ...  44.280
Estonia                                 NaN  ...  44.810
Finland                              44.415  ...  45.060
France                               51.150  ...  44.790
Germany                              51.340  ...  45.940
Greece                                  NaN  ...  44.730
Hong Kong                               NaN  ...  45.720
Hungary                                 NaN  ...  44.495
Iceland                                 NaN  ...  44.980
India                                   NaN  ...  44.680
Iran                                    NaN  ...  44.285
Ireland                                 NaN  ...  44.665
Israel                               55.755  ...  50.450
Italy                                47.750  ...  45.055
Japan                                57.100  ...  44.710
Lebanon                                 NaN  ...  44.655
Lithuania                               NaN  ...  44.355
Malaysia                                NaN  ...  44.815
Mexico                                  NaN  ...  45.080
Netherlands                          48.735  ...  47.110
New Zealand                             NaN  ...  44.880
Norway                               46.100  ...  46.140
Poland                                  NaN  ...  44.380
Portugal                                NaN  ...  44.745
Puerto Rico                             NaN  ...  44.175
Romania                                 NaN  ...  44.040
Russia                               54.190  ...  45.140
Saudi Arabia                            NaN  ...  44.440
Serbia                                  NaN  ...  44.420
Singapore                            53.440  ...  49.350
Slovak Republic                         NaN  ...  44.490
Slovenia                                NaN  ...  44.615
South Africa                            NaN  ...  46.090
South Korea                          51.310  ...  45.025
Spain                                   NaN  ...  44.610
Sweden                               51.340  ...  47.795
Switzerland                          51.465  ...  48.265
Taiwan                               54.210  ...  44.420
Thailand                                NaN  ...  44.825
Turkey                                  NaN  ...  44.415
USA                                  55.300  ...  46.980
Uganda                                  NaN  ...  44.280
United Arab Emirates                    NaN  ...  44.220
United Kingdom                       56.000  ...  45.900
Uruguay                                 NaN  ...  44.255
All                                  53.705  ...  45.100

[60 rows x 5 columns]

In [ ]:
#Las tablas dinamicas no se limitan a una sola funcion que desee aplicar. Se puede pasar un parametro llamado aggfunc,
#El cual sea una lista de diferentes funciones que desee aplicar. Y Pandas proveera un resultado usando nombres de las 
#columnas jerarquicas. A continuacion una la misma consulta con la funcion max() tambien.

df.pivot_table(values='score',index='country',columns='Rank_Level', aggfunc=[np.mean,np.max]).head()

mean  ...                   amax
Rank_Level First Tier Top Ranking  ... Third Tier Top Ranking
country                            ...                       
Argentina                     NaN  ...                    NaN
Australia                 47.9425  ...                  47.47
Austria                       NaN  ...                  47.78
Belgium                   51.8750  ...                  47.14
Brazil                        NaN  ...                    NaN

[5 rows x 8 columns]

In [ ]:
#Tambien se pueden resumir valores dentro de otra columna de nivel superior. Por ejemplo, si se quisiera ver un promedio
#general, y se quisiera ver el maximo, se puede indicar a Pandas lo haga proporcionando valores marginales.

df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean,np.max],margins=True).head()

mean  ...   amax
Rank_Level First Tier Top Ranking  ...    All
country                            ...       
Argentina                     NaN  ...  45.66
Australia                 47.9425  ...  51.61
Austria                       NaN  ...  47.78
Belgium                   51.8750  ...  52.03
Brazil                        NaN  ...  49.82

[5 rows x 10 columns]

In [ ]:
# creemos un dataFrame nuevo a partir del dataFrame anterior

new_df= df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean,np.max],margins=True)

print(new_df.index)
print(new_df.columns)

Index(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria',
       'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Iran',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia',
       'Serbia', 'Singapore', 'Slovak Republic', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand',
       'Turkey', 'USA', 'Uganda', 'United Arab Emirates', 'United Kingdom',
       'Uruguay', 'All'],
      dtype='object', name='country')
MultiIndex([('mean',  'First Tier Top Ranking'),
            ('mean',    'Other Top University'),
            ('mean', 'Second Tier Top Ranking'),
            ('mean',  'Third Tier To

In [ ]:
#Como vemos hay multi indexacion en las columnas, Top level contiene dos categorias mean y max.
#¿Como se consultaria esto?obtener el promedio de puntaje del First Tier Top University levels.
#Para lograrlo se necetaría proyectar dos DataFrames, el primero para la media, y el segundo para
#el Top Tier

new_df['mean']['First Tier Top Ranking'].head()

country
Argentina        NaN
Australia    47.9425
Austria          NaN
Belgium      51.8750
Brazil           NaN
Name: First Tier Top Ranking, dtype: float64

In [ ]:
#Ahora que sucedería si se quisiera encontrar el pais con la puntuacion media maxima en el nivel
#universitario del primer nivel?. Se puede utilizar la funcion idmax()
new_df['mean']['First Tier Top Ranking'].idxmax()

'United Kingdom'

In [ ]:
#Si se desea una forma diferente de visualizar la tabla dinamica se puede realizar con las funciones
#pila(stack()) y desapilamiento(unstack()). El apilamiento es girar el indice de la columna mas bajo.
#para convertirse en el indice más interno. El desaapilamiento es girar el indice de la columna mas arriba.
#para convertirse en el indice más externo. 

#revisems una vez mas nuestra tabla dinamica

new_df.head()

mean  ...   amax
Rank_Level First Tier Top Ranking  ...    All
country                            ...       
Argentina                     NaN  ...  45.66
Australia                 47.9425  ...  51.61
Austria                       NaN  ...  47.78
Belgium                   51.8750  ...  52.03
Brazil                        NaN  ...  49.82

[5 rows x 10 columns]

In [ ]:
#Intentemos primero con stack

new_df=new_df.stack()
new_df.head()

mean   amax
country   Rank_Level                               
Argentina Other Top University     44.672857  45.66
          All                      44.672857  45.66
Australia First Tier Top Ranking   47.942500  51.61
          Other Top University     44.645750  45.97
          Second Tier Top Ranking  49.242500  50.40

In [ ]:
#Ahora Unstack

new_df.unstack().head()

mean  ...    amax
Rank_Level First Tier Top Ranking  ...     All
country                            ...        
All                     58.350675  ...  100.00
Argentina                     NaN  ...   45.66
Australia               47.942500  ...   51.61
Austria                       NaN  ...   47.78
Belgium                 51.875000  ...   52.03

[5 rows x 10 columns]

In [ ]:
#Que pasaria si lo hacemos dos veces seguidas?
new_df.unstack().unstack().head()


      Rank_Level              country  
mean  First Tier Top Ranking  All          58.350675
                              Argentina          NaN
                              Australia    47.942500
                              Austria            NaN
                              Belgium      51.875000
dtype: float64

# Data/Time Funcionality

En esta sección se veran series de tiempo y funcionalidad de fecha en pandas. Manipular fechas, y horas en Pandas es bastante flexible, por lo cual permite realizar mas analisis como el analisis de *series de tiempo*. En realidad Pandas fue originalmente creado por Wed McKinney para manejar datos de fecha y hora

## Timestamp

Pandas tiene **cuatros** clases principales relacionadas con **Time**. Timestamp, DatetimeIndex, Period, y PeriodIndex. Comencemos con Timestamp.

Representa una unica marca de tiempo, y valores asociados con puntos de tiempo.

In [ ]:
import pandas as pd
import numpy as np
#Creemos un timestamp con el string 9/1/2019 10:05 AM. y aquí tenemos nuestro Timestamp. 
#Timestamp es intercambiable con la hora y fecha de Python en la mayoría de los casos.

pd.Timestamp('9/1/2019 10:05AM')

Timestamp('2019-09-01 10:05:00')

In [ ]:
#Tambien se puede crear un timestamp(marca de tiempo) pasando varios parametros como año, mes, dia
#hora y minuto por separado. 
pd.Timestamp(2019,12,20,0,0)

Timestamp('2019-12-20 00:00:00')

In [ ]:
#Timestamp tiene algunos atributos utiles, como isoweekday() el cual muestra el dia de la semana correspondiente
#con 1 como lunes y 7 como domingo 
pd.Timestamp(2019,12,20,0,0).isoweekday()

5

In [ ]:
#tambien se puede sacar especificamente año,mes,dia....

pd.Timestamp(2019,12,20,5,2,23).second

23

## Periodo

Hay situaciones en las que en realidad no se quiere trabajar con un momento en especifico, sino con un lapso de tiempo, la clase Period sirve para ello. Period representa un periodo de tiempo unico, como un dia o mes especifico

In [ ]:
#Vamos a crear un periodo solo para el mes de Enero.

pd.Period('1/2016')

Period('2016-01', 'M')

In [ ]:
#La 'M' representa la granularidad del periodo que indica que es un Mes(Month). Aqui otro ejeplo

pd.Period('3/5/2016')

Period('2016-03-05', 'D')

In [ ]:
#La aritmetica con este tipo de datos es sencilla, por ejemplo si queremos ver 5 meses despues sumamos 5
# en el caso donde la granularidad es 1

pd.Period('1/2016')+5

Period('2016-06', 'M')

In [ ]:
# O Si se quiere dos dias antes
pd.Period('3/5/2016') - 2

Period('2016-03-03', 'D')

### DatetimeIndex y PeriodIndex

In [ ]:
#Estos son muy utiles cuando se comienza a trabajar con series de tiempo
#Primero se creará un Timestamp(marca de tiempo) de los dias  1-3 de septiembre del 2016, cada Timestamp
#es el indice y tiene un valor asociado, en este aso a,b,c 

t1=pd.Series(list('abc'), [pd.Timestamp('2016-09-01'),pd.Timestamp('2016-09-02'),pd.Timestamp('2016-09-03')])
t1

2016-09-01    a
2016-09-02    b
2016-09-03    c
dtype: object

In [ ]:
#Si se observan los indices, se denota que tenemos DatetimeIndex.
type(t1.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [ ]:
#De manera similar se realiza con PeriodIndex

t2=pd.Series(list('abc'), [pd.Period('2016-09'),pd.Period('2016-10'),pd.Period('2016-11')])
t2

2016-09    a
2016-10    b
2016-11    c
Freq: M, dtype: object

In [ ]:
type(t2.index)

pandas.core.indexes.period.PeriodIndex

### Convirtiendo a Datetime

Suponiendo que se tiene una lista de fechas como strings, y se quiere crear un DataFrame

In [ ]:
#probemos con varios formatos de fecha

d1=['2 June 2013','Aug 29, 2014','2015-06-26','7/12/16']

#Y ahora crear unos datos al azar 

ts3= pd.DataFrame(np.random.randint(10,100,(4,2)), index=d1,columns=['a','b'])

ts3

,a,b
2 June 2013,63,89
"Aug 29, 2014",62,11
2015-06-26,70,87
7/12/16,27,47


In [ ]:
#Usando Pandas to_datetime, Pandas tratará de convertir todos en un formato estandar
ts3.index=pd.to_datetime(ts3.index)
ts3

,a,b
2013-06-02,63,89
2014-08-29,62,11
2015-06-26,70,87
2016-07-12,27,47


In [ ]:
#to_datetime tambien tiene opciones que permiten cambiar el orden de analisis de fecha. Por ejemplo,
# si el parametro dayfirst=True, para fecha Europea. 
pd.to_datetime('4.7.12',dayfirst=True)

Timestamp('2012-07-04 00:00:00')

### Timedelta

Timedelta son diferencias de tiempo. No es lo mismo que un periodo, aunque lo parezcan, conceptualmente son diferentes. Por ejemplo, la diferencia entre septiembre 3 y 1. 

In [ ]:
pd.Timestamp('9/3/2016')-pd.Timestamp('9/1/2016')

Timedelta('2 days 00:00:00')

In [ ]:
#Tambien se puede hacer mas preciso
pd.Timestamp('9/3/2016 8:10AM')+pd.Timedelta('12D 3H')

Timestamp('2016-09-15 11:10:00')

## Offset

Offset es muy similar a Timedelta, pero sigue reglas especificas de duración de calendario. Offset permite flexibilidad en terminos de intervalos de tiempo. Ademas de hora, día, semana, mes etc. tambien cosas como día de trabajo, y fin de mes, semi mes, etc. 

In [ ]:
pd.Timestamp('9/4/2016').weekday()

6

In [ ]:
#Ahora se puede añadir un timestamp con una semana por delante.

pd.Timestamp('9/4/16')+ pd.offsets.Week()

Timestamp('2016-09-11 00:00:00')

In [ ]:
#Ahora realicemos el mismo procedimiento con un fin de mes
pd.Timestamp('9/4/16')+pd.offsets.MonthEnd()


Timestamp('2016-09-30 00:00:00')

## Trabajando con dates(fechas) en un DataFrame

Ahora un par de trucos para trabajar con fechas en un DataFrame. Supongamos que se quiere mirar nueve medidas, tomadas dos por semanas, todos los domingos, a partir del octubre de 2016. Usando date_range, se puede crear un DatatimeIndex. En data_range se debe especificar la fecha de inicio, y finalización. Si no se especifica de manera de predeterminada, por defecto, la fecha presente se considera la fecha de inicio. Se debe fijar un numero de periodos, y frecuencia. Se fijara '2W-SUN'.

In [ ]:
dates=pd.date_range('10-01-2016', periods=9, freq='2W-SUN')
dates

DatetimeIndex(['2016-10-02', '2016-10-16', '2016-10-30', '2016-11-13',
               '2016-11-27', '2016-12-11', '2016-12-25', '2017-01-08',
               '2017-01-22'],
              dtype='datetime64[ns]', freq='2W-SUN')

In [ ]:
#Hay otro tipos de frecuencias que se pueden utilizar, como puede ser el dia habil(business day)

pd.date_range('10-01-2016',periods=9, freq='B')

DatetimeIndex(['2016-10-03', '2016-10-04', '2016-10-05', '2016-10-06',
               '2016-10-07', '2016-10-10', '2016-10-11', '2016-10-12',
               '2016-10-13'],
              dtype='datetime64[ns]', freq='B')

In [ ]:
#otro común es el trimestra, comenzando en Junio.

pd.date_range('04-01-2016',periods=12,freq='QS-JUN')

DatetimeIndex(['2016-06-01', '2016-09-01', '2016-12-01', '2017-03-01',
               '2017-06-01', '2017-09-01', '2017-12-01', '2018-03-01',
               '2018-06-01', '2018-09-01', '2018-12-01', '2019-03-01'],
              dtype='datetime64[ns]', freq='QS-JUN')

In [ ]:
#Volvamos al primer ejemplo con los domingos y con unos datos aleatorios.

dates=pd.date_range('10-01-2016', periods=9, freq='2W-SUN')

df=pd.DataFrame({'Cont 1': 100 + np.random.randint(-5,10,9).cumsum(),
                 'Cont 2': 120 + np.random.randint(-5,10,9)}, index=dates)
df

,Cont 1,Cont 2
2016-10-02,103,122
2016-10-16,102,124
2016-10-30,110,127
2016-11-13,108,116
2016-11-27,114,126
2016-12-11,112,122
2016-12-25,119,126
2017-01-08,123,126
2017-01-22,124,117


In [ ]:
#Podemos revisar que efectivamente todas las fechas sean un domingo
df.index.weekday

Int64Index([6, 6, 6, 6, 6, 6, 6, 6, 6], dtype='int64')

In [ ]:
#Se puede usar diff, para hallar la diferencia entre cada valor 
df.diff()

,Cont 1,Cont 2
2016-10-02,NaN,NaN
2016-10-16,-1.0,2.0
2016-10-30,8.0,3.0
2016-11-13,-2.0,-11.0
2016-11-27,6.0,10.0
2016-12-11,-2.0,-4.0
2016-12-25,7.0,4.0
2017-01-08,4.0,0.0
2017-01-22,1.0,-9.0


In [ ]:
#Suponiendo que se desea saber el conteo medio para cada mes en el dataFrame, se puede realizar
#usando resample(). Convirtiendo de una frecuencia mas alta a una más baja. 

df.resample('M').mean()

,Cont 1,Cont 2
2016-10-31,105.0,124.333333
2016-11-30,111.0,121.000000
2016-12-31,115.5,124.000000
2017-01-31,123.5,121.500000


In [ ]:
#Ahora hablemos de indexación (indexing) y recortes(slicing) en datetime. Por ejemplo, se puede
#usar la indexación parcial de strings para encontrar valores de un año en particular. 
df['2017']

,Cont 1,Cont 2
2017-01-08,123,126
2017-01-22,124,117


In [ ]:
# O tambien de un mes en particular

df['2016-12']

,Cont 1,Cont 2
2016-12-11,112,122
2016-12-25,119,126


In [ ]:
#o incluso desde una fecha en particular

df['2016-12':]

,Cont 1,Cont 2
2016-12-11,112,122
2016-12-25,119,126
2017-01-08,123,126
2017-01-22,124,117
